In [1]:
import tensorflow as tf
from numpy.random import RandomState
tf.logging.set_verbosity(tf.logging.ERROR)

In [2]:
batch_size = 8

w1 = tf.Variable(tf.random_normal([2,3], stddev=1, seed=1))
w2 = tf.Variable(tf.random_normal([3,1], stddev=1, seed=1))

In [3]:
x = tf.placeholder(tf.float32, shape=(None, 2), name='x-input')
y_ = tf.placeholder(tf.float32, shape=(None, 1), name='y-input')

In [10]:
a = tf.matmul(x, w1)
y = tf.matmul(a, w2)
y = tf.sigmoid(y)

In [5]:
cross_entropy = -tf.reduce_mean(
    y_ * tf.log(tf.clip_by_value(y, 1e-10, 1.0))
    +(1-y_)*tf.log(tf.clip_by_value(1-y, 1e-10, 1.0))
)
train_step = tf.train.AdamOptimizer(0.001).minimize(cross_entropy)

In [6]:
rdm = RandomState(1)
dataset_size = 128
X = rdm.rand(dataset_size, 2)
Y = [[int(x1+x2) < 1] for [x1, x2] in X]

In [11]:
with tf.Session() as sess:
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    print(sess.run(w1))
    print(sess.run(w2))
    
    STEPS = 5000
    for i in range(STEPS):
        start = (i * batch_size) % dataset_size
        end = min(start+batch_size, dataset_size)
        
        sess.run(train_step, feed_dict={x: X[start:end], y_: Y[start:end]})
        
        if i % 1000 == 0:
            total_corss_entropy = sess.run(cross_entropy, feed_dict={x:X, y_:Y})
            
            print(f'After {i} training step(s), cross entropy on all data is {total_corss_entropy}')
    print(sess.run(w1))
    print(sess.run(w2))

[[-0.8113182   1.4845988   0.06532937]
 [-2.4427042   0.0992484   0.5912243 ]]
[[-0.8113182 ]
 [ 1.4845988 ]
 [ 0.06532937]]
After 0 training step(s), cross entropy on all data is 3.2831037044525146
After 1000 training step(s), cross entropy on all data is 1.1156255006790161
After 2000 training step(s), cross entropy on all data is 1.0901048183441162
After 3000 training step(s), cross entropy on all data is 1.0900802612304688
After 4000 training step(s), cross entropy on all data is 1.0900791883468628
[[-0.21047649  0.8565735   1.0136005 ]
 [-2.218695   -0.15080711  0.86432105]]
[[-0.50675327]
 [ 0.8442636 ]
 [-0.5358139 ]]


RuntimeError: Attempted to use a closed Session.